# Lab 1

In [1]:
def qubic_spline_coeff(x_nodes, y_nodes):
    n = len(x_nodes)
    a = list(y_nodes)
    h = [x_nodes[i + 1] - x_nodes[i] for i in range(n - 1)] 
    
    alph = [3 / h[i] * (a[i + 1] - a[i]) 
         - 3 / h[i - 1] * (a[i] - a[i - 1]) for i in range(1, n - 1)]
    
    l = [1 for i in range(n)]
    mu = [0 for i in range(n - 1)]
    z = [0 for i in range(n)]
    
    for i in range(1, n - 1):
        l[i] = 2 * (x_nodes[i + 1] - x_nodes[i - 1]) - h[i - 1] * mu[i - 1]
        mu[i] = h[i] / l[i] 
        z[i] = (alph[i - 1] - h[i - 1] * z[i - 1]) / l[i]
        
    c = [0 for i in range(n)]
    b = [0 for i in range(n - 1)]
    d = [0 for i in range(n - 1)]
    
    for j in range(n - 1, 0, -1):
        c[j - 1] = z[j - 1] - mu[j - 1] * c[j]
        b[j - 1] = ((a[j] - a[j - 1]) / h[j - 1] 
                    - h[j - 1] * (c[j] + 2 * c[j - 1]) / 3)
        d[j - 1] = (c[j] - c[j - 1]) / 3 / h[j - 1]
        
    c.pop(-1)
    a.pop(-1)
    
    matr = np.c_[a, b, c, d]
    
    return matr

In [2]:
def qubic_spline(x, x_nodes, qs_coeff):
    
    for i in range(len(x_nodes) - 1):
        if x <= x_nodes[i + 1]:
            break
            
    return (qs_coeff[i][0] + qs_coeff[i][1] * (x - x_nodes[i]) 
            + qs_coeff[i][2] * (x - x_nodes[i]) ** 2 
            + qs_coeff[i][3] * (x - x_nodes[i]) ** 3)

In [3]:
def d_qubic_spline(x, x_nodes, qs_coeff):
    
    for i in range(len(x_nodes) - 1):
        if x <= x_nodes[i+1]:
            return (qs_coeff[i][1] + 2 * qs_coeff[i][2] * (x - x_nodes[i]) 
                    + 3 * qs_coeff[i][3] * (x - x_nodes[i]) ** 2)
            
    return None

In [4]:
def l_i(i, x, x_nodes):
    n = len(x_nodes)
    p = 1
    
    for j in range(n):
        if j != i:
            p *= (x - x_nodes[j]) / (x_nodes[i] - x_nodes[j])
            
    return p

In [5]:
def L(x, x_nodes, y_nodes):
    n = len(x_nodes)
    s = 0
    
    for i in range(n):
        s += y_nodes[i] * l_i(i, x, x_nodes)
    
    return s

In [6]:
def conf_band(y, n):
    y_ = y.transpose()
    for i in range(n + 1):
        y_[i].sort()
        
    h_l = [(y_[i][48] + y_[i][49]) / 2 for i in range(n + 1)]
    h_m = [sum(y_[i]) / 1000 for i in range(n + 1)]
    h_u = [(y_[i][948] + y_[i][949]) / 2 for i in range(n + 1)]
    
    return [h_l, h_m, h_u]

# Lab 3

In [7]:
def euler(x_0, t_n, f, h):
    n = len(f) - 1
    w = [x_0]
    
    for t in np.arange(h, t_n + h, h):
        w.append([])
        for j in range(n):
            w[-1].append(w[-2][j] + h * f[j](t, w[-2]))
        
        w[-1] = f[-1](w[-1])
        
    return w

In [8]:
def implicit_euler(x_0, t_n, f, h):
    n = len(f) - 1
    w = [x_0]

    def sys_eq(eq):
        x = eq
        w_ = [x[i] - w[-1][i] - h * f[0](t, x) for i in range(n)]
        return w_
    
    for t in np.arange(h, t_n + h, h):
        x_0 = root(sys_eq, w[-1], method = 'df-sane').x
        w.append([])  
        for i in range(n):      
            w[-1].append(w[-2][i] + h * f[i](t, x_0))
            
        w[-1] = f[-1](w[-1])
         
    return w

In [9]:
def runge_kutta(x_0, t_n, f, h):
    n = len(f) - 1
    w = [x_0]
    
    for t in np.arange(h, t_n + h, h):
        k1 = np.array([h * f[i](t, w[-1]) for i in range(n)])
        k2 = np.array([h * f[i](t + h / 2, [x + y / 2 for x, y in zip(w[-1], k1)])
                       for i in range(n)])
        k3 = np.array([h * f[i](t + h / 2, [x + y / 2 for x, y in zip(w[-1], k2)]) 
                       for i in range(n)])
        k4 = np.array([h * f[i](t + h, [x + y for x, y in zip(w[-1], k3)]) 
                       for i in range(n)])
        w.append(list([x + (p + 2*y + 2*z + q) / 6 
                  for x, p, y, z, q in zip(w[-1], k1, k2, k3, k4)]))
        
        w[-1] = f[-1](w[-1])
    
    return w

# Lab 4

In [10]:
def pca(A):
    m = len(A)
    K = A.T @ A 
    
    w, v = np.linalg.eig(K)
    
    idx = w.argsort()[::-1]   
    w = w[idx]
    v = v[:,idx]
    
    sigma = sqrt(w) / sqrt(m - 1)
    return(v.T, sigma)

In [11]:
def laplac(G):
    D = []
    for g in G:
        D.append(sum(g))
    return np.diag(D, 0) - G

In [12]:
def spetrum(G):
    eigenValues, eigenVectors = np.linalg.eig(G)
    
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    
    return(eigenVectors.T, eigenValues)